In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge

In [2]:
data = pd.read_csv('energydata_complete.csv')

In [3]:
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


#### From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [4]:
X = data['T2'].values.reshape(-1, 1)
y = data['T6'].values.reshape(-1, 1)

In [5]:
lin  = LinearRegression()
lin.fit(X,y)
pred = lin.predict(X)

In [6]:
round(lin.score(X,y),2)

0.64

In [7]:
df = data.drop(columns=['date','lights'])
df.columns

Index(['Appliances', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
       'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [8]:
scaler = MinMaxScaler()

In [9]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [10]:
features_df = normalised_df.drop(columns=['Appliances'])
target = normalised_df[ 'Appliances' ]

In [11]:
features_df.columns

Index(['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5',
       'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1',
       'rv2'],
      dtype='object')

In [12]:
x_train, x_test, y_train, y_test = train_test_split(features_df, target,test_size= 0.3 , random_state= 42 )

In [13]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

##### What is the Mean Absolute Error (in two decimal places)?

In [14]:
mae = mean_absolute_error(y_test, predicted_values)
round(mae,2)

0.05

#### What is the Residual Sum of Squares (in two decimal places)?

In [15]:
rss = np.sum(np.square(y_test - predicted_values))
round(rss,2)

45.35

#### What is the Root Mean Squared Error (in three decimal places)?

In [16]:
linrmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(linrmse,3)

0.088

#### What is the Coefficient of Determination (in two decimal places)?

In [17]:
r2_score = r2_score(y_test, predicted_values)
round(r2_score,2) 

0.15

In [18]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
ridge_pred = ridge_reg.predict(x_test)

In [19]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [20]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train,y_train)
lasso_pred = lasso_reg.predict(x_test)

In [21]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_Weight')

In [22]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on = 'Features')


#### Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [23]:
final_weights[final_weights.Linear_Model_Weight == final_weights.Linear_Model_Weight.min()][['Features','Linear_Model_Weight']]

,Features,Linear_Model_Weight
0,RH_2,-0.456698


In [24]:
final_weights[final_weights.Linear_Model_Weight == final_weights.Linear_Model_Weight.max()][['Features','Linear_Model_Weight']]

,Features,Linear_Model_Weight
25,RH_1,0.553547


#### Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [25]:
np.count_nonzero(final_weights.Lasso_Weight)

4

#### What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [26]:
rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
round(rmse,3)

0.094

In [27]:
rd_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
round(rd_rmse,3)

0.088

#### Train a ridge regression model with an alpha value of 0.4.Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [31]:
round(linrmse,3) == round(rd_rmse,3)

True